# Коллокации и Pointwise Mutual Information (PMI)

**Коллокация** - сочетание из двух или более слов, которое:

1. обладает некомпозициональной семантикой.
2. состоит из двух слов, которые встречаются вместе значимо часто.

При лингвистическом анализе текста коллокации нам очень нужны по понятным причинам:

* выявить, о чем текст
* найти различия между текстами на схожую тематику
* находить упоминания каких-то событий и персон
* и т.д.

Значит, нам нужно уметь выделять их автоматически. Чтобы найти коллокации, состоящие из двух слов, нам нужно выделить такие биграммы, которые встречаются друг с другом значимо часто, по сравнению с их индивидуальными частотами.

Для выделения коллокаций существуют разные метрики. Мы познакомимся с одной из них, которая называется **[Pointwise Mutual Information (PMI)](https://en.wikipedia.org/wiki/Pointwise_mutual_information)**. Формула рассчета PMI - 

$$ pmi = log \frac{{p(x,y)}}{p(x) \cdot p(y)} $$

* x и y - слова, входящие в биграмму;
* p(x,y) - вероятность появления биграммы x + y;
* p(x) и p(y) -- вероятность появления каждого из элементов биграммы в отдельности.

Значит, чтобы посчитать PMI, нам нужно:

* взять текст
* сделать частотный список всех слов в тексте
* найти вероятность появления каждого слова в тексте, разделив его частотность на общее количество слов
* сделать частотный список всех биграмм в тексте
* найти вероятность появления каждой биграммы в тексте, разделив её частотность на общее количество биграмм (а сколько их?)
* найти pmi по формуле
* биграммы с наибольшим pmi - коллокации в этом тексте.

Давайте попробуем:

In [1]:
import re
from math import log

punct = '[.,!«»?&@"$\[\]\(\):;%#&\'—-]'

def preprocessing(text): # функция предобработки текста
    text_wo_punct = re.sub(punct, '', text.lower()) # удаляем пунктуацию, приводим в нижний регистр
    words = text_wo_punct.strip().split() # делим по пробелам
    return words

In [2]:
with open('../news.txt', 'r', encoding='utf-8') as f:
    words = preprocessing(f.read())

word_freq = {} # создаем частотный словарь для слов
for word in words:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1

bigrams = [] # в цикле собираем биграммы из двух следующих друг за другом слов
for ind in range(1, len(words) - 1):
    bigrams.append(' '.join([words[ind - 1], words[ind]]))
    
bigram_freq = {} # создаем частотный словарь для биграмм
for b in bigrams:
    if b in bigram_freq:
        bigram_freq[b] += 1
    else:
        bigram_freq[b] = 1

In [3]:
def count_pmi(x, y): # Вычисляем pmi
    p_xy = bigram_freq[' '.join([x, y])]/len(bigrams) # считаем вероятность появления в тексте биграммы
    p_x, p_y = word_freq[x]/len(words), word_freq[y]/len(words) # считаем вероятность появления в тексте слов по отдельности
    pmi = log(p_xy/(p_x * p_y)) # по форму вычисляем pmi
    return pmi

In [4]:
pmi = {}
for bigr in bigrams: # для всех биграмм вычисляем pmi
    x, y = bigr.split()
    pmi[bigr] = count_pmi(x, y)

i = 0
for bigram in sorted(pmi, key = lambda m: -pmi[m]): # сортируем по убыванию коэффициента, распечатываем 100 биграмм 
    if i > 100:                                     # с самым большим коэффициентом
        break
    print(bigram, pmi[bigram])
    i += 1

курсообразование чревато 12.07619172480532
межведомственного электронного 12.07619172480532
депозитарных расписок 12.07619172480532
вызвавшие возражение 12.07619172480532
плавающего операционного 12.07619172480532
бинарных опционов 12.07619172480532
макаронами консервами 12.07619172480532
утратили обязательность 12.07619172480532
привлекла двухлетний 12.07619172480532
укрнафты днепроазотом 12.07619172480532
контейнерными поездами 12.07619172480532
сдвиг периодически 12.07619172480532
дискретного аукциона 12.07619172480532
относительной открытости 12.07619172480532
оградиться стеной 12.07619172480532
встречающие прорвали 12.07619172480532
выстраивать оборонную 12.07619172480532
излишки вагонного 12.07619172480532
гарегин тосунян 12.07619172480532
вячеславу жарову 12.07619172480532
валютное рефинансирование 12.07619172480532
мусоровозы прицепили 12.07619172480532
распространился поверх 12.07619172480532
option premium 12.07619172480532
николас папаниколау 12.07619172480532
продиктовано о

Какие слова имеют большой коэффициент pmi, а какие маленький?

С помощью pmi можно также искать биграммы, специфичные для какой-то категории текстов. Для этого нам нужно всего лишь посчитать коэффициент PMI для слова/биграммы и категории.

Например, у нас есть тексты 3 категорий. Посчитаем для слов в этих текстах коэффициент PMI, при этом в формуле X будет словом, а Y - категорией. (X, Y) - сколько раз слово X встретилось в текстах категории Y.

In [5]:
import os # собираем тексты, раскладываем их по категориям
anek = ''
teh = ''
izvest = ''
for root, dirs, files in os.walk('texts'):
    for f in files:
        if 'anekdots' in root:
            num_anek = len(files)
            anek += open(os.path.join(root, f)).read()
        elif 'izvest' in root:
            num_izvest = len(files)
            izvest += open(os.path.join(root, f)).read()
        elif 'teh_mol' in root:
            num_teh = len(files)
            teh += open(os.path.join(root, f)).read()
            
words_anek = preprocessing(anek) # пропроцессинг
words_teh = preprocessing(teh)
words_izvest = preprocessing(izvest)

words = words_anek + words_teh + words_izvest # в массиве words - весь наш корпус

In [6]:
def freq_dict(arr): # функция создания частотного словаря
    dic = {}
    for element in arr:
        if element in dic:
            dic[element] += 1
        else:
            dic[element] = 1
    return dic

In [7]:
corpus_freq = freq_dict(words) # считаем частотные словари для каждой категории в отдельности и для всего корпуса
anek_freq = freq_dict(words_anek)
izvest_freq = freq_dict(words_izvest)
teh_freq = freq_dict(words_teh)

In [10]:
def pmi_for_cats(x, y): # вычисляем pmi для слова и категории
    if y == 'anek': # определяем, что за категория нам требуется, задаем её переменные (массив слов, число текстов)
        dic = anek_freq
        arr = words_anek
        num = num_anek
    elif y == 'teh':
        dic = teh_freq
        arr = words_teh
        num = num_teh
    elif y == 'izvest':
        dic = izvest_freq
        arr = words_izvest
        num = num_izvest
    p_xy = dic[x]/len(arr)
    p_x, p_y = corpus_freq[x]/len(words), num/(num_izvest + num_teh + num_anek)
    pmi = log(p_xy/(p_x * p_y))
    return pmi

In [11]:
cat_pmi = {}
i = 0
for word in corpus_freq:
    if i > 100:
        break
    try:
        pmi_anek = pmi_for_cats(word, 'anek')
    except KeyError:
        pmi_anek = 0
    try:
        pmi_teh = pmi_for_cats(word, 'teh')
    except KeyError:
        pmi_teh = 0
    try:
        pmi_izvest = pmi_for_cats(word, 'izvest')
    except KeyError:
        pmi_izvest = 0
    max_pmi = max(pmi_anek, pmi_teh, pmi_izvest)
    if max_pmi == 0:
        continue
    if max_pmi == pmi_anek:
        cat = 'anek'
    elif max_pmi == pmi_teh:
        cat = 'teh'
    elif max_pmi == pmi_izvest:
        cat = 'izvest'
    print(word, cat)
    i += 1

каптерке anek
судостроители teh
молодчина anek
позволили izvest
сфокусировать anek
120 anek
сник teh
молодежь anek
внутреннюю teh
жидкость teh
нарисуй anek
считанные teh
помыл anek
мышеловку anek
болель anek
уходящих izvest
констанции anek
швеция teh
разрядить anek
расплатиться anek
убедить izvest
частоколом anek
курнем anek
полупромышленную teh
однокривошипных teh
сохранился teh
тащит anek
появившийся izvest
проверял anek
дивидендов anek
классическими izvest
научнотехническая teh
зажаренную anek
настеньку anek
зашвырнул izvest
малышку anek
гулянки anek
домогается anek
котеночка anek
ключито anek
договорятся anek
обрушившимся teh
ванная anek
укладывались teh
восстановлению izvest
лунном teh
выстроив anek
чувствует anek
назовете teh
бутылках anek
может izvest
бравый teh
многообразии teh
аккумуляторыто anek
рейки teh
государственников izvest
запахло anek
пальцем anek
мао anek
батенева izvest
глобализации izvest
опору teh
microsoft izvest
приводили teh
древних teh
штанишках anek
открывай 

## Задания

1. Отфильтровать слова, для которых мы вычисляем категории: убрать короткие и служебные, взять самые частотные.
2. Изменить алгоритм вычисления категорий, в качестве P(x) взяв не частоту слова во всем корпусе, а частоту слова в текстах всех остальных категорий кроме той, для которой мы сейчас вычисляем PMI: то есть для pmi_anek - частоту слова в текстах teh_mol и izvest, для pmi_teh - частоту слова в текстах anek и izvest, и т.д.
3. Соотнести биграммы с категориями.